# Assignments and Real Problems
Practice problems based on real-world patterns. Try to solve without looking up answers.

## Assignment 1: Data normalization (tabular)
Given raw features, standardize each column to zero mean and unit variance.

In [6]:
from numpy import mean
import torch

# Example data: rows are samples, columns are features
X = torch.tensor([
    [1.0, 200.0, 0.5],
    [2.0, 180.0, 0.3],
    [3.0, 210.0, 0.9],
    [4.0, 190.0, 0.1],
])

# TODO: standardize columns
# mean = ?
# std = ?
# X_std = ?

mean = X.mean(dim=0) # what is this? dim = 0 means we are computing the mean for each feature (column)
std = X.std(dim=0) # what is this? dim = 0 means we are computing the std for each feature (column)
X_std = (X - mean) / std

# Print standardized data
print(X_std)

tensor([[-1.1619,  0.3873,  0.1464],
        [-0.3873, -1.1619, -0.4392],
        [ 0.3873,  1.1619,  1.3175],
        [ 1.1619, -0.3873, -1.0247]])


## Assignment 2: Linear regression for house prices
Predict house price from size and number of rooms using a linear model.

In [11]:
from math import e
from requests import get
import torch
import torch.nn as nn

# Features: [size (sq ft), rooms]
X = torch.tensor([
    [800.0, 2.0],
    [1000.0, 3.0],
    [1200.0, 3.0],
    [1500.0, 4.0],
    [1800.0, 4.0],
])

# Prices in thousands
Y = torch.tensor([[150.0], [200.0], [230.0], [280.0], [320.0]])

# TODO: build a linear model, train for 200 epochs, print final loss
# model = ?
# criterion = ?
# optimizer = ?

class SimpleLinearRegression(nn.Module):
    def __init__(self,input_dim, output_dim):
        super().__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.linear(x)

model = SimpleLinearRegression(input_dim=2, output_dim=1)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

epochs = 200
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X)
    loss = criterion(outputs, Y)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 20 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

# Save the model
torch.save(model.state_dict(), "simple_linear_regression.pth")
# what is pth?
# .pth is a file extension used by PyTorch to indicate a file that contains a serialized model or tensor.
# use it to save and load models
# load to load the model
# model = SimpleLinearRegression(input_dim=2, output_dim=1) # why this ? We need to create a new instance of the model to load the saved parameters into.
model.load_state_dict(torch.load("simple_linear_regression.pth")) # what is this? This loads the model's parameters from the specified file.

def get_parameters(model):
    return {name: param.data for name, param in model.named_parameters()}

get_parameters(model)

Epoch [20/200], Loss: 6827.9741
Epoch [40/200], Loss: 927.8101
Epoch [60/200], Loss: 151.1384
Epoch [80/200], Loss: 120.3104
Epoch [100/200], Loss: 92.8200
Epoch [120/200], Loss: 92.0170
Epoch [140/200], Loss: 91.9770
Epoch [160/200], Loss: 91.9053
Epoch [180/200], Loss: 91.8610
Epoch [200/200], Loss: 91.8205


{'linear.weight': tensor([[0.1851, 0.3658]]), 'linear.bias': tensor([-0.3361])}

## Assignment 3: Logistic regression (spam classifier)
Classify messages as spam/non-spam using bag-of-words features.

In [11]:
import torch
import torch.nn as nn

# Input features (bag-of-words)
x = torch.tensor([
    [1, 0, 0, 1, 0],
    [0, 1, 0, 1, 1],
    [0, 0, 1, 0, 0],
    [1, 1, 0, 0, 0],
    [0, 0, 1, 1, 1],
    [1, 0, 1, 0, 1],
], dtype=torch.float32)

# Labels
y = torch.tensor([[1.0], [1.0], [0.0], [0.0], [1.0], [0.0]])

model = nn.Linear(5, 1)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

for epoch in range(100):
    logits = model(x)
    loss = criterion(logits, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Prediction
logits = model(x)
probabilities = torch.sigmoid(logits)

print(probabilities)


tensor([[0.6896],
        [0.8300],
        [0.3426],
        [0.2655],
        [0.7557],
        [0.2516]], grad_fn=<SigmoidBackward0>)


## Assignment 4: Real problem - time series smoothing
Given a noisy sensor signal, compute a moving average and visualize trends.

In [15]:
import torch

# Simulated sensor data
signal = torch.tensor([3.0, 4.1, 5.2, 4.9, 4.0, 3.8, 3.2, 2.9, 3.1, 3.7])

# TODO: compute moving average with window size 3
# Hint: use convolution or manual slicing
window_size=3
moving_average=[]
for i in range(len(signal)-window_size+1):
    moving_average.append(signal[i:i+window_size].mean())
    
    
moving_average=torch.tensor(moving_average)
    
print(moving_average)


tensor([4.1000, 4.7333, 4.7000, 4.2333, 3.6667, 3.3000, 3.0667, 3.2333])


## Assignment 5: Real problem - simple image edge detection
Use a Sobel-like filter to detect edges in a tiny grayscale image.

In [23]:
import torch
import torch.nn.functional as F

# 5x5 grayscale image
img = torch.tensor([
    [10, 10, 10, 10, 10],
    [10, 20, 20, 20, 10],
    [10, 20, 50, 20, 10],
    [10, 20, 20, 20, 10],
    [10, 10, 10, 10, 10],
], dtype=torch.float32)

# Sobel-like horizontal kernel
kernel = torch.tensor([
    [-1, -2, -1],
    [ 0,  0,  0],
    [ 1,  2,  1]
], dtype=torch.float32)

# PyTorch conv2d expects 4D tensors: (batch, channels, height, width)
img_4d = img.view(1, 1, 5, 5)      # 1 image, 1 channel
kernel_4d = kernel.view(1, 1, 3, 3) # 1 filter, 1 channel

# Apply convolution
edges = F.conv2d(img_4d, kernel_4d, padding=1)  # padding=1 to keep size same
edges = edges.squeeze()  # remove batch and channel dims

print("Detected edges:")
print(edges)


Detected edges:
tensor([[  40.,   70.,   80.,   70.,   40.],
        [  10.,   60.,  100.,   60.,   10.],
        [   0.,    0.,    0.,    0.,    0.],
        [ -10.,  -60., -100.,  -60.,  -10.],
        [ -40.,  -70.,  -80.,  -70.,  -40.]])


## Assignment 6: Real problem - predicting energy usage
Given hourly temperature and occupancy, predict energy usage.

In [37]:
import torch
import torch.nn as nn

# Input features: [temperature, occupancy]
X = torch.tensor([
    [18.0, 0.0],
    [20.0, 1.0],
    [22.0, 1.0],
    [24.0, 1.0],
    [26.0, 0.0],
    [28.0, 0.0],
])

# Target energy usage
Y = torch.tensor([[8.0], [10.0], [12.0], [14.0], [9.0], [7.0]])

# Define a small MLP: 2 inputs → 4 hidden → 1 output
class EnergyMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.input_to_hidden = nn.Linear(2, 4)   # input → hidden layer
        self.activation = nn.ReLU()              # activation function
        self.hidden_to_output = nn.Linear(4, 1)  # hidden → output layer

    def forward(self, x):
        x = self.input_to_hidden(x)  # input → hidden
        x = self.activation(x)       # apply activation
        x = self.hidden_to_output(x) # hidden → output
        return x

# Create model
model = EnergyMLP()

# Loss and optimizer
loss_function = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# Training loop
for epoch in range(500):
    predicted = model(X)                  # forward pass
    loss = loss_function(predicted, Y)    # compute loss

    optimizer.zero_grad()  # reset gradients
    loss.backward()        # backpropagation
    optimizer.step()       # update weights

print(f"Final Loss = {loss.item():.2f}")

# Predictions
predicted_energy = model(X)
print("\nPredicted Energy Usage:")
print(predicted_energy)


Final Loss = 5.67

Predicted Energy Usage:
tensor([[9.9996],
        [9.9996],
        [9.9996],
        [9.9996],
        [9.9996],
        [9.9996]], grad_fn=<AddmmBackward0>)


## Optional challenge
- Implement a custom Dataset and DataLoader for any assignment above.
- Add a validation split and report validation loss.

In [54]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset,DataLoader, random_split

# 1️⃣ Dataset
features = torch.tensor([
    [1,0,0,1,0],   # sample 1
    [0,1,0,1,1],   # sample 2
    [0,0,1,0,0],   # sample 3
    [1,1,0,0,0],   # sample 4
    [0,0,1,1,1],   # sample 5
    [1,0,1,0,1]    # sample 6
], dtype=torch.float32)

labels = torch.tensor([[1],[1],[0],[0],[1],[0]], dtype=torch.float32)

dataset=TensorDataset(features,labels)
train_set,val_set=random_split(dataset,[4,2])
train_loader=DataLoader(train_set,batch_size=2,shuffle=True)
val_loader=DataLoader(val_set,batch_size=2)

model=nn.Linear(5,1)
criterion=nn.BCEWithLogitsLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=0.1)

for epoch in range(200):
    for x_batch,y_batch in train_loader:
        loss=criterion(model(x_batch),y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
with torch.no_grad():
    val_loss = sum(criterion(model(vx), vy).item() for vx,vy in val_loader)/len(val_loader)
print("Validation Loss:", val_loss)


with torch.no_grad():
    probabilities=torch.sigmoid(model(features))
print("Predictied Probabilities:",probabilities)
    

Validation Loss: 0.2790018916130066
Predictied Probabilities: tensor([[0.9257],
        [0.8929],
        [0.1154],
        [0.0642],
        [0.9401],
        [0.3590]])
